# 1. 환경설정

### engine, session_maker, base 생성

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

#reference: https://docs.sqlalchemy.org/en/14/orm/session_api.html#session-and-sessionmaker
engine = create_engine('sqlite:///example.db', echo=True)
Session = sessionmaker(engine)
Base = declarative_base()

/tmp/ipykernel_94633/3416448916.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Entity 생성

In [2]:
from sqlalchemy import Column, Integer, String

class Demo(Base):
    __tablename__ = 'demos'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

In [3]:
for k,v in Base.metadata.tables.items():
    print(f"{k}: {v}")

demos: demos


In [4]:
Base.metadata.create_all(engine)

2024-06-29 01:36:18,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-29 01:36:18,825 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("demos")
2024-06-29 01:36:18,827 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-29 01:36:18,834 INFO sqlalchemy.engine.Engine COMMIT


# 2. session autoflush True(default) 테스트

In [5]:
session = Session()

In [6]:
session.autoflush

True

In [7]:
# 객체 생성
demo = Demo(name="hello", password="world")
session.add(demo)

In [8]:
# select * from Demo
result = session.query(Demo).all()
for r in result:
    print(f"{r.name}: {r.password}")

2024-06-29 01:36:32,095 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-29 01:36:32,098 INFO sqlalchemy.engine.Engine INSERT INTO demos (name, password) VALUES (?, ?)
2024-06-29 01:36:32,100 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('hello', 'world')
2024-06-29 01:36:32,111 INFO sqlalchemy.engine.Engine SELECT demos.id AS demos_id, demos.name AS demos_name, demos.password AS demos_password 
FROM demos
2024-06-29 01:36:32,112 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()
hello: world
hello: world
hello: world


### session.commit을 하지 않았으므로 session이 닫히는 순간 transaction rollback 수행

In [9]:
session.close()

2024-06-29 01:36:43,817 INFO sqlalchemy.engine.Engine ROLLBACK


# 2. session autoflush False 테스트
* reference: https://stackoverflow.com/questions/4201455/sqlalchemy-whats-the-difference-between-flush-and-commit

In [10]:
session_autoflush_false = Session()
session_autoflush_false.autoflush = False

In [11]:
session_autoflush_false.autoflush

False

In [12]:
demo2 = Demo(name="hello2", password="world2")
session_autoflush_false.add(demo2)

### flush가 false로 설정되어 있어 demo2 ORM이 조회가 안됨

In [13]:
result2 = session_autoflush_false.query(Demo).all()
for r in result2:
    print(f"{r.name}: {r.password}")    

2021-08-19 16:44:12,352 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-19 16:44:12,353 INFO sqlalchemy.engine.Engine SELECT demos.id AS demos_id, demos.name AS demos_name, demos.password AS demos_password 
FROM demos
2021-08-19 16:44:12,354 INFO sqlalchemy.engine.Engine [cached since 0.07028s ago] ()


In [14]:
session_autoflush_false.flush()

2021-08-19 16:44:12,367 INFO sqlalchemy.engine.Engine INSERT INTO demos (name, password) VALUES (?, ?)
2021-08-19 16:44:12,367 INFO sqlalchemy.engine.Engine [cached since 0.08762s ago] ('hello2', 'world2')


### flush함수를 호출해서 트랙잰션 생셩

In [15]:
result2 = session_autoflush_false.query(Demo).all()
for r in result2:
    print(f"{r.name}: {r.password}")  

2021-08-19 16:44:12,381 INFO sqlalchemy.engine.Engine SELECT demos.id AS demos_id, demos.name AS demos_name, demos.password AS demos_password 
FROM demos
2021-08-19 16:44:12,382 INFO sqlalchemy.engine.Engine [cached since 0.09909s ago] ()
hello2: world2


In [16]:
session_autoflush_false.close()

2021-08-19 16:47:10,807 INFO sqlalchemy.engine.Engine ROLLBACK
